# Functions

In [255]:
def analyze_match(match):
    points = match["pbp"]
    pbp_id = match["pbp_id"]
    points += "."
    data = {}

    # server1, server2
    servers = [1, -1]
    # serves won, serves total, service games won, service games total, sets won, tiebreaks won
    metrics = ["points", "games", "sets"]
    
    for server in servers:
        for metric in metrics:
            data[(server, metric)] = 0
    
    p = 0
    server = 1
    # points so far, p1 points, p2 points, p1 games, p2 games, p1 sets, p2 sets, server
    # pbp_id, data[(1,"points")], data[(-1,"points")], data[(1,"games")], data[(-1,"games")], data[(1,"sets")], data[(-1,"sets")], server
    point_by_point = np.array([[pbp_id, 0, 0, 0, 0, 0, 0, 1]])
    last_point_winner = 0
    while p < len(points):
        if points[p] == "S" or points[p] == "A":
            current_score = [pbp_id, data[(1,"points")], data[(-1,"points")], data[(1,"games")], data[(-1,"games")], data[(1,"sets")], data[(-1,"sets")], server]
            point_by_point = np.vstack([point_by_point, current_score])
            # server won the point
            data[(server, "points")] += 1
            last_point_winner = server
            if data[(server, "points")] == 4 and data[(server*-1, "points")] == 4:
                data[(server, "points")] = 3
                data[(server*-1, "points")] = 3
        elif points[p] == "R" or points[p] == "D":
            current_score = [pbp_id, data[(1,"points")], data[(-1,"points")], data[(1,"games")], data[(-1,"games")], data[(1,"sets")], data[(-1,"sets")], server]
            point_by_point = np.vstack([point_by_point, current_score])
            # returner won the point
            data[(server*-1, "points")] += 1
            last_point_winner = server * -1
            if data[(server, "points")] == 4 and data[(server*-1, "points")] == 4:
                data[(server, "points")] = 3
                data[(server*-1, "points")] = 3
        elif points[p] == ";":
            # end of the game
            if last_point_winner == server:
                data[(server, "games")] += 1
            else:
                data[(server*-1, "games")] += 1
            data[(server, "points")] = 0
            data[(server*-1, "points")] = 0
            server *= -1
            
        elif points[p] == ".":
            # set finished, not in tiebreak
            if last_point_winner == server:
                data[(server, "sets")] += 1
            else:
                data[(server * -1, "sets")] += 1
            data[(server, "points")] = 0
            data[(server*-1, "points")] = 0
            data[(server, "games")] = 0
            data[(server*-1, "games")] = 0
            server *= -1
            

            
        elif points[p] == "/":
            # Entered tiebreak
            next_set_server = server * -1
            while p < len(points) and points[p] != ".":
                if points[p] == "/":
                    server *= -1
                elif points[p] == "S" or points[p] == "A":
                    current_score = [pbp_id, data[(1,"points")], data[(-1,"points")], data[(1,"games")], data[(-1,"games")], data[(1,"sets")], data[(-1,"sets")], server]
                    point_by_point = np.vstack([point_by_point, current_score])
                    
                    # server won the point
                    data[(server, "points")] += 1
                    last_point_winner = server
                    if data[(server, "points")] == 7 and data[(server*-1, "points")] == 7:
                        data[(server, "points")] = 6
                        data[(server*-1, "points")] = 6
                elif points[p] == "R" or points[p] == "D":
                    current_score = [pbp_id, data[(1,"points")], data[(-1,"points")], data[(1,"games")], data[(-1,"games")], data[(1,"sets")], data[(-1,"sets")], server]
                    point_by_point = np.vstack([point_by_point, current_score])
                    
                    data[(server*-1, "points")] += 1
                    last_point_winner = server * -1
                    if data[(server, "points")] == 7 and data[(server*-1, "points")] == 7:
                        data[(server, "points")] = 6
                        data[(server*-1, "points")] = 6
                p += 1
            
            data[(last_point_winner, "sets")] += 1
            data[(server, "points")] = 0
            data[(server*-1, "points")] = 0
            data[(server, "games")] = 0
            data[(server*-1, "games")] = 0
            server = next_set_server
        p += 1
        
    
    
    return point_by_point[1:]

In [256]:
from scipy.special import loggamma
from scipy.optimize import fsolve


def sigmoid(theta, x):
    return 1 / (1 + math.e**(-theta.dot(x)))

def compute_lloss(theta, X, y):
    res = 0
    for i in range(len(y)):
        if y[i] == 1:
            res += np.log(sigmoid(theta, X[i]))
        else:
            res += np.log(1 - sigmoid(theta, X[i]))
            
    return (-res / len(y))

def compute_current_objective_value(theta, X, y, penal):
    res = 0
    for i in range(len(y)):
        if y[i] == 1:
            res += np.log(sigmoid(theta, X[i]))
        else:
            res += np.log(1 - sigmoid(theta, X[i]))
            
    return (-res / len(y)) + (penal * np.linalg.norm(theta, ord=1))

def backtracking_line_search(theta, gamma, grad, X, y, penal):
    

    
    alpha=1
    tau = 0.6
    c = 0.05
    t = -c*gamma.dot(grad)
    current_obj = compute_current_objective_value(theta,X,y,penal)
    print(f"current_obj: {current_obj}")
    print(f"theta: {theta}")
    while True:
        candidate_obj = compute_current_objective_value(theta+(alpha*gamma), X,y,penal)
        print(f"theta: {theta+(alpha*gamma)}")
        print(f"alpha: {alpha}")
        print(f"candidate_obj: {candidate_obj}, current_obj: {current_obj}")
        if current_obj - candidate_obj >= alpha*t - 0.000000001:
            break
        alpha = tau*alpha
    return alpha
        

def irls_lars(matches, feature_cols, outcome_col, max_iterations, tol, penal):
    
    theta = np.array([0]*len(feature_cols))
    X = matches[feature_cols].to_numpy()
    # X_T = X.transpose()
    y = matches[outcome_col].to_numpy()
    prev_objective_value = None
    
    for _ in range(max_iterations): 
        print()
        print(f"iteration {_}")
        # compute lambda_(i,i) and z_i
        diag_vals = []
        y_tilde = []
        X_tilde = []
        grad = [0]*len(feature_cols)
        for i in range(len(X)):
            lambda_ii = sigmoid(theta, X[i]) * (1 - sigmoid(theta, X[i]))
            z_i = sigmoid(theta, X[i]) + ((1 - sigmoid(y[i]*theta, X[i]))*(y[i])) / lambda_ii
            
            y_tilde.append((lambda_ii**0.5) * z_i)
            X_tilde.append((lambda_ii**0.5)*X[i])
            grad += (1 - sigmoid(y[i]*theta,X[i]))*y[i]*X[i]
        
        y_tilde = np.array(y_tilde)
        X_tilde = np.array(X_tilde)
        
        reg = linear_model.Lasso(alpha=penal)
        reg.fit(X_tilde, y_tilde)
        gamma = np.array(reg.coef_)
        print(f"gamma: {gamma}")
        
        grad = np.array(grad)
        # do backtracking line search to move efficiently towards optimum theta
        alpha_line_search = backtracking_line_search(theta, gamma, grad, X, y, penal)
        print(f"alpha_line_search: {alpha_line_search}")
        # update theta
        theta = theta + alpha_line_search*gamma
        
        # check stopping criterion
        candidate_obj = compute_current_objective_value(theta, X, y,penal)
        if prev_objective_value is not None and np.abs(prev_objective_value - candidate_obj) < tol:
            return {"success": True, "obj": candidate_obj, "theta": theta}
        else:
            prev_objective_value = candidate_obj
            print(f"obj_val: {prev_objective_value}")
            print(f"theta: {theta}")
            
    return {"success": False, "obj": candidate_obj, "theta": theta}
        
        
        
        
        
            
        
    

In [257]:
import math
def number_win_in_k_most_recent_matches(match, k):
    a = float("nan")
    b = float("nan")
    
    if type(match["p1 year match results"]) == str and len(match["p1 year match results"]) >= k:
        i = 0
        a = 0
        while i < k:
            if match["p1 year match results"][len(match["p1 year match results"]) - 1 - i] == "W":
                a += 1
            i += 1
            
            
    if type(match["p2 year match results"]) == str and len(match["p2 year match results"]) >= k:
        i = 0
        b = 0
        while i < k:
            if match["p2 year match results"][len(match["p2 year match results"]) - 1 - i] == "W":
                b += 1
            i += 1
            
    return a,b
            
        
    

In [258]:
def generate_features(match, feature_cols):
    res = []
    for feature in feature_cols:
        feature_arr = feature.split("-")
        res.append(match[feature_arr[0]] - match[feature_arr[1]])
    return tuple(res)

In [7]:
np.diag([0,1,4]) ** 0.5

NameError: name 'np' is not defined

In [259]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import beta
from multiprocessing import Pool, freeze_support
import math
pd.set_option("display.max_columns", None)

In [41]:
# import the markov model functions lazily
%run Modeling_tennis_match_iid_points.ipynb

0.69881
0.933139


In [42]:
matches = pd.read_csv("./data/2011-2015_matches_cleaned_forShrinkage.csv")
matches["date"] = matches.apply(lambda x: datetime.strptime(x["date"], '%Y-%m-%d'), axis=1)
matches

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6,6,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2229262,2011-07-28,Credit,ATP,Main,Matthias Bachinger,Julien Benneteau,2,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,0,57,36,59,7,10,0,0,43,58,9,10,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [43]:
# Calculate tournament averages
matches["tourney serve win rate"] = matches["tourney serves won"] / matches["tourney serves total"]
matches["tourney return win rate"] = 1 - matches["tourney serve win rate"]
matches["tourney service game win rate"] = matches["tourney service games won"] / matches["tourney service games total"]
matches["tourney return game win rate"] = 1 - matches["tourney service game win rate"]

In [44]:
# compute sets needed to win match (to use later in row filtering for evaluation)
# Computing sets needed to win the match
matches["tourney sets to win"] = matches[["p1 sets won", "p2 sets won"]].max(axis=1)

# compute number of wins in k most recent matches
k = 5
k_most_recent = [f"p1 wins in {k} most recent matches", f"p2 wins in {k} most recent matches"]
matches[k_most_recent] = matches.apply(lambda match: number_win_in_k_most_recent_matches(match,k), axis=1, result_type="expand")

# change value of winner
matches["winner"].replace(2,0, inplace=True)
matches["p1 year serve total"].replace(0, float("nan"), inplace=True)
matches["p2 year serve total"].replace(0, float("nan"), inplace=True)
matches["p1 year return total"].replace(0, float("nan"), inplace=True)
matches["p2 year return total"].replace(0, float("nan"), inplace=True)
matches["p1 year service game total"].replace(0, float("nan"), inplace=True)
matches["p2 year service game total"].replace(0, float("nan"), inplace=True)
matches["p1 year return game total"].replace(0, float("nan"), inplace=True)
matches["p2 year return game total"].replace(0, float("nan"), inplace=True)
matches["p1 year set total"].replace(0, float("nan"), inplace=True)
matches["p2 year set total"].replace(0, float("nan"), inplace=True)
matches["p1 year tiebreak total"].replace(0, float("nan"), inplace=True)
matches["p2 year tiebreak total"].replace(0, float("nan"), inplace=True)

8222
2386


In [37]:
# create big dataset with one row for each point in each match
matches_pbp = None
for index, row in matches.iterrows():
    match = analyze_match(row)
    match_df = pd.DataFrame(
        {
            'pbp_id': match[:, 0], 
            'p1 points': match[:, 1],
            'p2 points': match[:,2],
            'p1 games': match[:,3],
            'p2 games': match[:,4],
            'p1 sets': match[:,5],
            'p2 sets': match[:,6],
            'server': match[:,7],
        }
    )
    if matches_pbp is None:
        matches_pbp = match_df
    else:
        matches_pbp = matches_pbp.append(match_df, ignore_index=True)

In [38]:
matches_pbp

,pbp_id,p1 points,p2 points,p1 games,p2 games,p1 sets,p2 sets,server
0,2231275,0,0,0,0,0,0,1
1,2231275,1,0,0,0,0,0,1
2,2231275,2,0,0,0,0,0,1
3,2231275,3,0,0,0,0,0,1
4,2231275,0,0,1,0,0,0,-1
...,...,...,...,...,...,...,...,...
1718534,8634739,1,0,5,6,1,1,1
1718535,8634739,1,1,5,6,1,1,1
1718536,8634739,1,2,5,6,1,1,1
1718537,8634739,2,2,5,6,1,1,1


In [39]:
all_match_data_pbp = pd.merge(matches, matches_pbp, on='pbp_id', how='left')

In [40]:
all_match_data_pbp

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,tourney sets to win,p1 points,p2 points,p1 games,p2 games,p1 sets,p2 sets,server
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,1
1,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,0,0,0,0,1
2,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,0,0,0,0,0,1
3,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,0,0,0,0,0,1
4,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,1,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718534,8634739,2015-12-20,Men'sS,ATP,Main,Matwe Middelkoop,Robin Haase,0,SSSS;SSSS;SRRSRSSS;SSRRRSRSSS;SSRSRS;SRSSS;RSS...,7-6(4) 2-6 7-5,0,121,63,97,13,16,1,0

In [41]:
all_match_data_pbp.to_csv("./data/all_match_data_pbp.csv", encoding='utf-8', index=False)

In [74]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import beta
from multiprocessing import Pool, freeze_support
import math
from sklearn import linear_model
pd.set_option("display.max_columns", None)

In [4]:
all_match_data_pbp = pd.read_csv("./data/all_match_data_pbp.csv")
all_match_data_pbp["date"] = all_match_data_pbp.apply(lambda x: datetime.strptime(x["date"], '%Y-%m-%d'), axis=1)

In [5]:
#training_matches = all_match_data_pbp[all_match_data_pbp["date"] <= datetime.strptime("31 Dec 14", '%d %b %y')].copy()
#test_matches = all_match_data_pbp[all_match_data_pbp["date"] > datetime.strptime("31 Dec 14", '%d %b %y')].copy()  

In [123]:
# generate feature columns
feature_cols = ["p1 serve win rate-p2 serve win rate", "p1 return win rate-p2 return win rate", "p1 opp serve win rate-p2 opp serve win rate", "p1 opp return win rate-p2 opp return win rate", "p1 year serve total-p2 year serve total", "p1 year return total-p2 year return total"]
matches[feature_cols] = matches.apply(lambda match: generate_features(match, feature_cols), axis=1, result_type="expand")

# standardize feature columns
for feature in feature_cols:
    matches[feature] = (matches[feature] - matches[feature].mean()) / matches[feature].std()

matches


,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust,p1 wins in 5 most recent matches,p2 wins in 5 most recent matches,p1 serve win rate-p2 serve win rate,p1 return win rate-p2 return win rate,p1 opp serve win rate-p2 opp serve win rate,p1 opp return win rate-p2 opp return win rate,p1 year serve total-p2 year serve total,p1 year return total-p2 year return total
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,0,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,0,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6,6,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,2,2,2,2,NaN,NaN,2,2,2,2,NaN,NaN,NaN

In [80]:
training_matches = matches[matches["date"] <= datetime.strptime("31 Dec 14", '%d %b %y')].copy()
test_matches = matches[(matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y'))&(matches["p1 match count"] >= 2) & (matches["p2 match count"] >= 2)].copy()                                                                                             
print(len(training_matches))
print(len(test_matches))

8222
2386


In [93]:
outcome_col = "winner"

training_matches = matches[feature_cols+[outcome_col]].dropna(subset=feature_cols)

In [94]:
training_matches

,p1 serve win rate-p2 serve win rate,p1 return win rate-p2 return win rate,p1 opp serve win rate-p2 opp serve win rate,p1 opp return win rate-p2 opp return win rate,p1 year serve total-p2 year serve total,p1 year return total-p2 year return total,winner
10,1.291256,-1.649587,3.685686,-0.495190,-0.112100,-0.115795,0
18,-3.536914,4.791103,-10.666936,2.414992,-0.121748,-0.111667,0
19,-2.329001,-2.142752,4.784671,7.076174,-0.085836,-0.098768,0
20,-2.329001,-2.142752,4.784671,7.076174,-0.085836,-0.098768,0
21,1.431689,3.198472,-7.117869,-4.645508,-0.123892,-0.112699,0
...,...,...,...,...,...,...,...
10781,-0.764246,-3.418759,4.640497,3.842069,-0.078868,-0.099284,0
10782,1.007709,-0.229176,-0.313467,-0.279218,0.168229,0.137549,1
10784,0.700691,3.731661,-7.542864,-0.597110,-1.627912,-1.548662,0
10785,0.717896,-0.260695,0.714826,-2.249064,-0.107812,-0.091544,0


In [152]:
# initialize parameters for irls_lars
max_iterations = 100000
tol = 10**(-7)
penal = 0.01

irls_lars(training_matches, feature_cols, outcome_col, max_iterations, tol,penal)


iteration 0
gamma: [0.11245228 0.09076981 0.07444079 0.         0.         0.23007089]
current_obj: 0.6931471805600548
theta: [0 0 0 0 0 0]
theta: [0.11245228 0.09076981 0.07444079 0.         0.         0.23007089]
alpha: 1
candidate_obj: 0.643338375750302, current_obj: 0.6931471805600548
alpha_line_search: 1
obj_val: 0.643338375750302
theta: [0.11245228 0.09076981 0.07444079 0.         0.         0.23007089]

iteration 1
gamma: [0.09053197 0.06992017 0.05894363 0.         0.         0.17436772]
current_obj: 0.643338375750302
theta: [0.11245228 0.09076981 0.07444079 0.         0.         0.23007089]
theta: [0.20298425 0.16068997 0.13338443 0.         0.         0.40443861]
alpha: 1
candidate_obj: 0.6235441556131486, current_obj: 0.643338375750302
alpha_line_search: 1
obj_val: 0.6235441556131486
theta: [0.20298425 0.16068997 0.13338443 0.         0.         0.40443861]

iteration 2
gamma: [0.07093835 0.05292837 0.04543813 0.         0.         0.13042258]
current_obj: 0.623544155613148

theta: [0.32280703 0.24860513 0.20952402 0.         0.         0.6214338 ]
alpha: 6.1886548790943145e-09
candidate_obj: 0.6178572221615013, current_obj: 0.6178572221562918
alpha_line_search: 6.1886548790943145e-09


{'success': True,
 'obj': 0.6178572221615013,
 'theta': array([0.32280703, 0.24860513, 0.20952402, 0.        , 0.        ,
        0.6214338 ])}

In [62]:
641781.2271335392

Index(['pbp_id', 'date', 'tny_name', 'tour', 'draw', 'server1', 'server2',
       'winner', 'pbp', 'score', 'adf_flag', 'wh_minutes', 'p1 serves won',
       'p1 serves total', 'p1 service games won', 'p1 service games total',
       'p1 sets won', 'p1 tiebreaks won', 'p2 serves won', 'p2 serves total',
       'p2 service games won', 'p2 service games total', 'p2 sets won',
       'p2 tiebreaks won', 'sets total', 'tourney serves won',
       'tourney serves total', 'tourney service games won',
       'tourney service games total', 'p1 serve win rate',
       'p1 return win rate', 'p1 service game win rate',
       'p1 return game win rate', 'p1 set win rate', 'p1 tiebreak win rate',
       'p2 serve win rate', 'p2 return win rate', 'p2 service game win rate',
       'p2 return game win rate', 'p2 set win rate', 'p2 tiebreak win rate',
       'p1 opp serve win rate', 'p1 opp return win rate',
       'p1 opp service game win rate', 'p1 opp return game win rate',
       'p1 opp set win r

In [160]:
outcome_col = "winner"
X = training_matches[feature_cols].to_numpy()
y = training_matches[outcome_col].to_numpy()

reg = linear_model.LogisticRegression(penalty='l1', solver='liblinear')
reg.fit(X, y)
theta = reg.coef_
print(theta)

[[ 0.42092467  0.38904918  0.29037626  0.19094746 -2.00095632  2.5491015 ]]


In [161]:
compute_lloss(theta, X, y)

array([0.59865647])

In [162]:
X = training_matches[feature_cols].to_numpy()
y = training_matches[outcome_col].to_numpy()
#theta = np.array([0.32280703, 0.24860513, 0.20952402, 0        , 0        ,0.6214338 ])
compute_lloss(theta, X, y)

array([0.59865647])

In [163]:
X = test_matches[feature_cols].to_numpy()
y = test_matches[outcome_col].to_numpy()
#theta = np.array([0.32280703, 0.24860513, 0.20952402, 0.        , 0.        ,0.6214338 ])
compute_lloss(theta, X, y)

array([0.58267853])